# Chapter 6: Deploy Models from Vertex AI Model Garden

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayoisio/genai-on-google-cloud/blob/main/chapter-6/colabs/02_model_garden_deployment.ipynb)

## Learning Goals

In this notebook, you will learn how to:
- Navigate and select models from Vertex AI Model Garden
- Deploy open-source models using vLLM serving containers
- Configure hardware based on model size requirements
- Send inference requests to deployed endpoints
- Manage costs by cleaning up resources

## Prerequisites

- A Google Cloud project with Vertex AI API enabled
- Familiarity with Python and the Google Cloud SDK
- (Optional) A HuggingFace account for gated model access

## 1. Setup and Authentication

In [ ]:
# Check environment
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")
    from google.colab import auth
    auth.authenticate_user()
    print("✓ Authentication successful!")
else:
    print("Running outside Colab - ensure gcloud is configured")

In [ ]:
# Install required packages
!pip install -q google-cloud-aiplatform>=1.50.0
print("✓ Packages installed!")

In [ ]:
# Configure project settings
import os

# Set your project details
PROJECT_ID = input("Enter your GCP Project ID: ")
REGION = input("Enter your region (e.g., us-central1): ") or "us-central1"

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID

print(f"\n✓ Project: {PROJECT_ID}")
print(f"✓ Region: {REGION}")

In [ ]:
# Initialize Vertex AI
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

print(f"✓ Vertex AI initialized")
print(f"  Project: {PROJECT_ID}")
print(f"  Region: {REGION}")

## 2. Understanding Model Garden

Vertex AI Model Garden provides access to:
- **First-party models**: Gemini, Gemma, PaLM (Google models)
- **Third-party models**: Llama, Mistral, Claude (partner models)
- **Open-source models**: Community models with various licenses

### Model Selection Framework

| Model Size | Use Case | GPU Requirement | Example |
|------------|----------|-----------------|----------|
| 2B-4B | Edge/Mobile, Low latency | 1x L4 (24GB) | Gemma 2B |
| 7B-9B | General purpose, Balance | 1x L4 or A100 | Gemma 7B, Llama 3 8B |
| 27B-70B | Complex reasoning | 2-4x A100 (80GB) | Gemma 27B, Llama 3 70B |
| 400B+ | Research, Max capability | TPU pods or 8x H100 | Llama 3 405B |

## 3. Deploy Gemma from Model Garden

We'll deploy Gemma 2 using the vLLM serving container for efficient inference.

In [ ]:
# Configuration for deployment
import datetime

# Model selection
MODEL_ID = "gemma-2-2b-it"  # Options: gemma-2-2b-it, gemma-2-9b-it, gemma-2-27b-it
HF_MODEL_ID = f"google/{MODEL_ID}"

# vLLM serving container
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20250116_0916_RC00"

# Hardware configuration based on model size
if "2b" in MODEL_ID:
    MACHINE_TYPE = "g2-standard-8"
    ACCELERATOR_TYPE = "NVIDIA_L4"
    ACCELERATOR_COUNT = 1
    GPU_MEMORY_UTILIZATION = 0.9
    MAX_MODEL_LEN = 8192
elif "9b" in MODEL_ID:
    MACHINE_TYPE = "g2-standard-24"
    ACCELERATOR_TYPE = "NVIDIA_L4"
    ACCELERATOR_COUNT = 2
    GPU_MEMORY_UTILIZATION = 0.9
    MAX_MODEL_LEN = 8192
else:  # 27b
    MACHINE_TYPE = "a2-highgpu-4g"
    ACCELERATOR_TYPE = "NVIDIA_A100_80GB"
    ACCELERATOR_COUNT = 4
    GPU_MEMORY_UTILIZATION = 0.95
    MAX_MODEL_LEN = 8192

print(f"Model: {MODEL_ID}")
print(f"Machine: {MACHINE_TYPE}")
print(f"Accelerator: {ACCELERATOR_COUNT}x {ACCELERATOR_TYPE}")
print(f"Max context length: {MAX_MODEL_LEN}")

In [ ]:
# Optional: Set HuggingFace token for gated models
# Some models require accepting license terms on HuggingFace

HF_TOKEN = input("Enter HuggingFace token (or press Enter to skip): ") or None

if HF_TOKEN:
    print("✓ HuggingFace token configured")
else:
    print("⚠ No HF token - using public model access")

In [ ]:
# Deploy the model
def deploy_model_vllm(
    model_name: str,
    model_id: str,
    machine_type: str,
    accelerator_type: str,
    accelerator_count: int,
    gpu_memory_utilization: float = 0.9,
    max_model_len: int = 4096,
):
    """Deploy a model using vLLM serving container."""
    
    # vLLM arguments
    vllm_args = [
        "python", "-m", "vllm.entrypoints.api_server",
        "--host=0.0.0.0",
        "--port=8080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        f"--gpu-memory-utilization={gpu_memory_utilization}",
        f"--max-model-len={max_model_len}",
        "--disable-log-stats",
    ]
    
    # Environment variables
    env_vars = {
        "MODEL_ID": model_id,
        "DEPLOY_SOURCE": "notebook",
    }
    
    if HF_TOKEN:
        env_vars["HF_TOKEN"] = HF_TOKEN
    
    # Create endpoint
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name}-endpoint",
        project=PROJECT_ID,
        location=REGION,
    )
    
    # Upload model
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_args=vllm_args,
        serving_container_ports=[8080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
        serving_container_deployment_timeout=7200,
        model_garden_source_model_name="publishers/google/models/gemma2",
    )
    
    print(f"Deploying {model_name} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)...")
    print("This may take 15-30 minutes...")
    
    # Deploy to endpoint
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
    )
    
    print(f"\n✓ Deployment complete!")
    print(f"  Endpoint: {endpoint.name}")
    
    return model, endpoint

In [ ]:
# Execute deployment
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"{MODEL_ID}-{timestamp}"

print("="*70)
print("DEPLOYING MODEL FROM MODEL GARDEN")
print("="*70)
print()

model, endpoint = deploy_model_vllm(
    model_name=model_name,
    model_id=HF_MODEL_ID,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
    max_model_len=MAX_MODEL_LEN,
)

## 4. Send Inference Requests

Now let's test our deployed model with some prompts.

In [ ]:
# Helper function for predictions
def generate_response(endpoint, prompt, max_tokens=256, temperature=0.7):
    """Generate a response from the deployed model."""
    
    # Format for vLLM
    instances = [{
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 0.9,
    }]
    
    response = endpoint.predict(instances=instances)
    
    return response.predictions[0]

In [ ]:
# Test prompts
test_prompts = [
    "Explain the concept of fine-tuning in machine learning in 3 sentences.",
    "What are the key differences between GPUs and TPUs for AI workloads?",
    "Write a Python function to calculate fibonacci numbers.",
]

print("="*70)
print("MODEL INFERENCE TESTS")
print("="*70)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Test {i} ---")
    print(f"Prompt: {prompt}")
    print()
    
    response = generate_response(endpoint, prompt)
    print(f"Response: {response}")
    print("-" * 70)

## 5. Monitor Endpoint Performance

Check the endpoint metrics and resource utilization.

In [ ]:
# Get endpoint details
print("="*70)
print("ENDPOINT INFORMATION")
print("="*70)
print()
print(f"Endpoint Name: {endpoint.display_name}")
print(f"Endpoint Resource: {endpoint.resource_name}")
print(f"Endpoint URI: {endpoint.name}")
print()

# List deployed models
for deployed_model in endpoint.list_models():
    print(f"Deployed Model ID: {deployed_model.id}")
    print(f"Model Display Name: {deployed_model.display_name}")

## 6. Clean Up Resources

**Important**: Delete resources to avoid ongoing charges. A deployed model can cost $1-5+ per hour depending on GPU type.

In [ ]:
# Cleanup - Uncomment to delete resources
cleanup = input("Delete deployed resources? (yes/no): ").lower()

if cleanup == "yes":
    print("\nCleaning up resources...")
    
    # Undeploy model from endpoint
    endpoint.undeploy_all()
    print("✓ Model undeployed")
    
    # Delete endpoint
    endpoint.delete()
    print("✓ Endpoint deleted")
    
    # Delete model
    model.delete()
    print("✓ Model deleted")
    
    print("\n✓ All resources cleaned up!")
else:
    print("\n⚠ Resources retained - remember to delete them later to avoid charges!")
    print(f"\nTo delete later, run:")
    print(f"  endpoint = aiplatform.Endpoint('{endpoint.resource_name}')")
    print(f"  endpoint.undeploy_all()")
    print(f"  endpoint.delete()")

## Summary

In this notebook, you learned how to:

1. **Navigate Model Garden** - Select models based on size, capability, and hardware requirements
2. **Deploy with vLLM** - Use optimized serving containers for efficient inference
3. **Configure hardware** - Match GPU resources to model size requirements
4. **Send requests** - Generate predictions from deployed endpoints
5. **Manage costs** - Clean up resources to avoid ongoing charges

### Cost Estimates

| Configuration | Hourly Cost (approx) |
|---------------|----------------------|
| 1x L4 (g2-standard-8) | ~$0.80-1.20/hour |
| 2x L4 (g2-standard-24) | ~$1.80-2.40/hour |
| 4x A100-80GB | ~$12-16/hour |

### Next Steps

- Try deploying different models (Llama 3, Mistral)
- Experiment with batch inference for higher throughput
- Explore the [Model Garden UI](https://console.cloud.google.com/vertex-ai/model-garden) for one-click deployments
- See **03_vllm_serving.ipynb** for advanced vLLM configuration